In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt


In [ ]:
# Reading data
file_path = "Weather Data.csv"  
weather_data = pd.read_csv(file_path)

In [ ]:
# Removing Unnecessary columns (2-features)
weather_data["Date/Time"] = pd.to_datetime(weather_data["Date/Time"])
weather_data = weather_data.drop(columns=["Date/Time"])  

In [ ]:
# Encoding "Weather" column
label_encoder = LabelEncoder()
weather_data["Weather"] = label_encoder.fit_transform(weather_data["Weather"])

In [ ]:
# Split into input and output
X = weather_data.drop(columns=["Weather"])
y = weather_data["Weather"]

In [ ]:
# Standardize input
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# one-hot encoding 
y = pd.get_dummies(y).values


In [ ]:
# Convert X into 3D (batch_size, timesteps, features), easier for CNN
X = X.reshape(X.shape[0], X.shape[1], 1)


In [ ]:
# Testing and Traing data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Constructing Model CNN
model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax') 
])

In [ ]:
# Compiling model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training model
history = model.fit(X_train, y_train, epochs=1000, batch_size=96, validation_data=(X_test, y_test), verbose=1)




In [ ]:

# Evaluating Model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Độ chính xác: {accuracy:.2f}")

# Plot
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid()
plt.show()
